In [3]:
import pandas as pd
from sqlalchemy import create_engine
import dotenv
import os

dotenv.load_dotenv(override=True)

PG_PASSWORD = os.getenv("PG_PASSWORD")
PG_USER = os.getenv("PG_USER")
PG_HOST = os.getenv("PG_HOST")
PG_PORT = os.getenv("PG_PORT")
PG_DATABASE = os.getenv("PG_DB")

# SQLAlchemy engine
engine = create_engine(
    f"postgresql+psycopg2://{PG_USER}:{PG_PASSWORD}@{PG_HOST}:{PG_PORT}/{PG_DATABASE}?sslmode=require"
)

# Create a connection object from the engine
conn = engine.connect()

ProgrammingError: (psycopg2.ProgrammingError) invalid dsn: invalid connection option "_l7YZ6S]S@fuji.ucll.be:52425/international_week?sslmode"

(Background on this error at: https://sqlalche.me/e/20/f405)

In [ ]:

def get_table_names():
    query = """
    SELECT table_name
    FROM information_schema.tables
    WHERE table_schema = 'public'
    """
    df = pd.read_sql(query, conn)
    return df["table_name"].tolist()

get_table_names()


NameError: name 'pd' is not defined

In [ ]:

def get_table_info(table_name):
    query = f"""
    SELECT column_name, data_type
    FROM information_schema.columns
    WHERE table_name = '{table_name}'
    """
    df = pd.read_sql(query, conn)
    return df

get_table_info("matches")

,column_name,data_type
0,match_id,character varying
1,match_date,timestamp without time zone
2,home_team_id,character varying
3,away_team_id,character varying
4,home_score,integer
5,away_score,integer


In [ ]:
def get_table_rows(table_name):
    query = f"SELECT * FROM {table_name}"
    df = pd.read_sql(query, conn)
    return df
get_table_rows("matches")


,match_id,match_date,home_team_id,away_team_id,home_score,away_score
0,5oc8drrbruovbuiriyhdyiyok,2024-07-26,1oyb7oym5nwzny8vxf03szd2h,b7jmo07lqav0wfe2mtlzyspak,1,1
1,5r7lyj3frtml3aqy458jrxj4k,2024-07-28,920raeoumft3q2wmvc296uceo,7zibyotq1x1hq6pmgibhr0e6s,0,1
2,5ptnar4qtaltvcfsjdw9vzhg4,2024-07-27,6vyr13j6kbwdtxjwm1m4m8nu2,2989v3kho9h5jaboe1nr7lx21,1,0
3,5qqz2do47zjms7rfrwzcths7o,2024-07-28,1uk609mhtsfhokhaewjapb26,8y3iucyxguipljcmf87a11bk9,0,1
4,5pcyhm34h5c948yji4oryevpw,2024-07-27,cyrrlv6l1onld5x247w1q1jlr,bw9wm8pqfzcchumhiwdt2w15c,0,0
...,...,...,...,...,...,...
141,77w03jwg8cs1dkqemcax540t0,2024-12-13,b7jmo07lqav0wfe2mtlzyspak,7zibyotq1x1hq6pmgibhr0e6s,1,1
142,789p8p9wchivsno0m437haeqc,2024-12-15,2989v3kho9h5jaboe1nr7lx21,6vyr13j6kbwdtxjwm1m4m8nu2,0,2
143,78nndczlbj2214zhtbfahs2s4,2024-12-22,ecj1gmc7pqcjo4sc9hvghj5u3,1oyb7oym5nwzny8vxf03szd2h,2,2
144,791cri1kfsl6c1h5ozv0iqcyc,2024-12-22,89xivtxk59uzvhdsd6miao6tm,6vyr13j6kbwdtxjwm1m4m8nu2,2,0


In [ ]:
tables = get_table_names()

# count all rows in all tables
def get_row_count(table_name):
    query = f"SELECT COUNT(*) FROM {table_name}"
    df = pd.read_sql(query, conn)
    return df.iloc[0, 0]
row_counts = {}

for table in tables:
    row_counts[table] = get_row_count(table)
row_counts
    

{'eventtypes': 36,
 'matches': 146,
 'matchevents': 218848,
 'player_tracking': 18510936,
 'players': 487,
 'qualifiers': 106049,
 'qualifiertypes': 41,
 'spadl_actions': 242450,
 'teams': 16,
 'role_passwords': 129,
 'player_position': 2963}

In [ ]:
# get all column names in all tables
query = f"""
SELECT column_name
FROM information_schema.columns
WHERE table_name = 'matchevents'
"""
df = pd.read_sql(query, conn)
df["column_name"].tolist()


['match_id',
 'event_id',
 'eventtype_id',
 'result',
 'success',
 'period_id',
 'timestamp',
 'end_timestamp',
 'ball_state',
 'ball_owning_team',
 'team_id',
 'player_id',
 'x',
 'y',
 'end_coordinates_x',
 'end_coordinates_y',
 'receiver_player_id']

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from scipy.spatial import ConvexHull
from mplsoccer import Pitch

# Database Query
query = """
SELECT 
    p.team_id, 
    p.player_id, 
    pt.frame_id, 
    pt.x, 
    pt.y, 
    t.team_name, 
    me.ball_owning_team
FROM matchevents me
INNER JOIN teams t ON me.ball_owning_team = t.team_id
INNER JOIN players p ON p.team_id <> me.ball_owning_team -- Exclude ball-owning players
INNER JOIN player_tracking pt 
    ON pt.player_id = p.player_id
    AND pt.period_id = me.period_id 
    AND ABS(EXTRACT(EPOCH FROM (pt."timestamp"::INTERVAL - me."timestamp"::INTERVAL))) <= 0.5  
WHERE me.match_id = '6c8lz4vpdfl8yxo9kgokz9250'
AND me.ball_owning_team <> 'bw9wm8pqfzcchumhiwdt2w15c'  
AND p.player_name <> 'Ball'
ORDER BY me."timestamp"
LIMIT 30
"""

# Execute query
conn = engine.connect()
df = pd.read_sql(query, conn)

# Select only a few frames (e.g., first 5 unique frames)
selected_frames = df['frame_id'].unique()[:5]

# Function to compute compactness metrics
def compute_compactness(frame_df):
    """ Computes spread, convex hull area, and centroid distance. """
    
    # Standard deviation of positions (spread of players)
    std_x = np.std(frame_df['x'])
    std_y = np.std(frame_df['y'])
    
    # Convex hull area (team shape)
    points = frame_df[['x', 'y']].to_numpy()
    hull = ConvexHull(points) if len(points) > 2 else None
    hull_area = hull.volume if hull else 0  

    # Centroid and average distance to centroid
    centroid = np.mean(points, axis=0)
    distances = np.linalg.norm(points - centroid, axis=1)
    mean_distance = np.mean(distances)
    
    return std_x, std_y, hull_area, mean_distance

# Compute compactness for selected frames
compactness_data = []
for frame_id in selected_frames:
    frame_df = df[df['frame_id'] == frame_id]
    std_x, std_y, hull_area, mean_distance = compute_compactness(frame_df)
    compactness_data.append([frame_id, std_x, std_y, hull_area, mean_distance])

compactness_df = pd.DataFrame(compactness_data, columns=['frame_id', 'std_x', 'std_y', 'hull_area', 'mean_distance'])

# Visualization using MLP Soccer
def plot_team_positions(frame_id, ax):
    """ Plots player positions and convex hull for a given frame """
    frame_df = df[df['frame_id'] == frame_id]
    ax.clear()
    
    pitch = Pitch(pitch_type='statsbomb', pitch_color='white', line_color='black')
    pitch.draw(ax=ax)

    # Plot player positions
    ax.scatter(frame_df['x'], frame_df['y'], s=200, edgecolors='black', color='blue', label='Players')

    # Draw convex hull
    points = frame_df[['x', 'y']].to_numpy()
    if len(points) > 2:
        hull = ConvexHull(points)
        for simplex in hull.simplices:
            ax.plot(points[simplex, 0], points[simplex, 1], color='red', linewidth=2)

    ax.set_title(f"Compactness - Frame {frame_id}")

# Create animation with only a few frames
fig, ax = plt.subplots(figsize=(8, 6))
pitch = Pitch(pitch_type='statsbomb', pitch_color='white', line_color='black')
pitch.draw(ax=ax)

def update(frame_id):
    plot_team_positions(frame_id, ax)

ani = animation.FuncAnimation(fig, update, frames=selected_frames, repeat=True)
writergif = animation.PillowWriter(fps=10)
ani.save('file.gif',writer=writergif)
plt.show()
